Solving the Schrodinger equation with scipy and interactive plotting with ipywidgets and matplotlib

In [ ]:
#import stuff
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, FloatSlider, fixed
from scipy.sparse.linalg import eigsh

We are going to find the energy eigenstates of the Schrodinger equation for 1-d potentials, so our equation to solve is:

$E\Psi(x) = \left(\frac{-\hbar}{2m}\frac{\mathrm{d}^2}{\mathrm{d}x^2} + V \right) \Psi(x)$

We can turn this into a discrete equation by approximating the second derivative:

$\frac{d^2f}{dx^2} \approx \frac{f(x+h) - 2 f(x) + f(x-h)}{h^2}$

which allows us to use a matrix eigenvalue solver to find out Energies (eigenvalues) and wavefunctions (eigenvectors). 

(we are also going to set $\frac{\hbar}{2m}=1$)

In [ ]:
def solve_potential(x, V, k=10):
    h = x[1] - x[0] #assume that the x array is evenly spaced
    m = len(x)
    # construct a matrix operator for the approximate second derivative
    difdif = h**(-2) * (
        -2 * np.diag(np.ones(m)) + 
        np.diag(np.ones(m-1), 1) + 
        np.diag(np.ones(m-1), -1)) 
    A = np.diag(V) - difdif 

    # 'SM' means that we want the k smallest eigenvalues
    energies, wavefunctions = eigsh(A, k=k, which='SM')
    return energies, wavefunctions

In [ ]:
def plot_wavefunctions(x, Psis, Es):
    #for clarity, we rescale the wavevectors
    norm = 0.9 / np.max(abs(Psis[:,0].real))
    for i, E in enumerate(Es):
        # flip signs so that we don't have any all-negative wavevectors
        if Psis[10,i].real < 0:
            Psis[:,i] = -Psis[:,i]
        # offset the wavevector by its energy
        plt.plot(x, norm*Psis[:,i].real + E.real, c='C0')

In [ ]:
def sqaure_potential_well(x, height, width):
    V = height * (np.heaviside(x - 0.5*width, 1) + np.heaviside(-0.5*width - x, 1))
    return V 

In [ ]:
def plot_psis_square(x, height, width):
    V = sqaure_potential_well(x, height, width)
    es, psis = solve_potential(x, V)
    plot_wavefunctions(x, psis, es)
    plt.plot(x, V, 'k')
    plt.ylim((-0.5,12))
    plt.xlim((-4,4))
    plt.xlabel(r'$x$')
    plt.show()

We have created a function 'plot_psis_square' which will solve the Schrodinger equation for a square potential well and then plot the result using matplotlib. Now we can interact with the function by using ipywidget sliders.

In [ ]:
interact(plot_psis_square, x=fixed(np.linspace(-5,5,100)),
                                   height=FloatSlider(10., min=0.1, max=20, continuous_update=False),
                                   width=FloatSlider(2.5, min=0.1, max=8., continuous_update=False))

we can do the same thing with any parameterised 1-d potential we want, such as polynomial potentials:

In [ ]:
def polynomial_potential(x, a, b, c, d):
    return a * x + b * x ** 2 + c * x ** 3 + d * x ** 4

In [ ]:
def plot_psis_poly(x, a, b, c, d):
    V = polynomial_potential(x, a, b, c, d)
    es, psis = solve_potential(x, V)
    plot_wavefunctions(x, psis, es)
    plt.plot(x, V, 'k')
    plt.ylim((-1.5,14))
    plt.xlim((-4,4))
    plt.title(r'$V=ax+bx^2+cx^3+dx^4$')
    plt.xlabel(r'$x$')
    plt.show()

In [ ]:
interact(plot_psis_poly, x=fixed(np.linspace(-4,4,100)),
                        a=FloatSlider(0.,min=-2,max=2, step=0.01, continuous_update=False),
                        b=FloatSlider(1.,min=-3., max=3., step=0.01, continuous_update=False),
                        c=FloatSlider(0., min=-1, max=1, step=0.01, continuous_update=False),
                        d=FloatSlider(0, min=-0.5, max=0.5, step=0.01, continuous_update=False))

feel free to mess around with it. Feedback welcome!